In [16]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gzip
import os
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
# Read all the article files (I think they're actually the whole newspaper per file)
cwd = os.getcwd()
data_dir = Path(cwd) / 'ldc_data' / 'data'
unparsed_article_sgml = []
for subdir in range(451, 504):
    curr_dir = data_dir / str(subdir)
    for zipped_file in os.listdir(curr_dir):
        with gzip.open(curr_dir / zipped_file, 'r') as f:
            unparsed_article_sgml.append(f.read())

unparsed_article_sgml[0]

b'<DOC>\n<SEQNO> 0002 </SEQNO>\n<MSGDATE> 19940722 </MSGDATE>\n<MSGTIME> 0204 </MSGTIME>\n<PRDSRVID> WJ CO </PRDSRVID>\n<DSPDATE> 19940722 </DSPDATE>\n<DSPTIME> 2305 </DSPTIME>\n<SOURCE> WJ   </SOURCE>\n<ACCESN> 199407220001 </ACCESN>\n<PAGE> B1          </PAGE>\n<CO> AAPL ASTA CPQ IBM INTC </CO>\n<IN> I/CPR I/MED I/PUB I/SEM </IN>\n<SB> N/HIY N/MRK N/WEI </SB>\n<MS> M/CYC M/TEC </MS>\n<RE> R/CA R/NME R/NY R/PRM R/TX R/US </RE>\n<HL> Computers:  PC Price War  May Break Out,  Starting in Fall  ---- \n     By Bart Ziegler  Staff Reporter of The Wall Street Journal </HL>\n<TEXT>\n<p>\n   NEW YORK -- A combination of factors in the personal computer industry is\nsetting the stage for a pricing free fall.  \n<p>\n   Unlike PC battles of the past, this one could be set off by structural\nchanges, and it could push prices down to the realm of camcorders and\nbig-screen TVs. "The industry is bracing itself," says Rick Inatome, chairman\nof computer distributor Inacom Corp.  \n<p>\n   Starting 

In [25]:
parsed_articles = []
for edition in unparsed_article_sgml:
    soup = BeautifulSoup(edition)
    for document in soup.find_all('doc'):
        for article in document.find_all('text'):
            parsed_articles.append(article.get_text().replace('\n', ''))

parsed_articles[0]

'   NEW YORK -- A combination of factors in the personal computer industry issetting the stage for a pricing free fall.     Unlike PC battles of the past, this one could be set off by structuralchanges, and it could push prices down to the realm of camcorders andbig-screen TVs. "The industry is bracing itself," says Rick Inatome, chairmanof computer distributor Inacom Corp.     Starting this fall, multimedia machines equipped with CD-ROM players andstereo sound could sell for less than $1,000, down about a third from today\'sprice for such a system. Since that steep decline would happen over just a fewmonths and would follow smaller cuts earlier this year, it would mark anacceleration in price-cutting, which typically runs 20% to 25% a year in the PCmarket.     Other PCs could drop to about $750, breaking the consumer-electronics pricebarrier and ending up on more back-to-school and Christmas lists.     Most PC makers won\'t discuss the possibility of a new round of price cuts.Privatel

In [39]:
df = pd.DataFrame(parsed_articles, columns=["text"])
df['num_sent'] = df.text.apply(lambda x : len(sent_tokenize(x)))

In [40]:
valid_articles = df[df['num_sent'] > 8]
valid_articles

,text,num_sent
0,NEW YORK -- A combination of factors in the...,51
1,NEW YORK -- The bond market was sluggish ye...,68
2,Tokyo stocks dropped broadly Thursday in ex...,46
3,DETROIT -- The view from Martin Zimmerman's...,37
5,AUCTION ACTION: Licenses sought for hand-he...,53
...,...,...
118832,When you have a goose that's laying golden ...,49
118834,NEW YORK -- A federal judge threw out the c...,27
118835,"YOUNGSTOWN, Ohio -- Phar-Mor Inc. said FoxM...",13
118836,CHINA TO REVISE GRAIN PRICES| China said...,35


In [41]:
valid_articles.to_csv('wsjArticles.csv')